# Maze Example

The creation of a maze implemented by applying multiple design patterns.

The maze is composed by rooms, bounded by walls and doors that estabilish the communication between two adjacent rooms.

## Base Classes

In [ ]:
from abc import ABC, abstractmethod
from enum import Enum

# Direction Enum
class Direction(Enum):
    NORTH = 0
    EAST = 1
    SOUTH = 2
    WEST = 3

# MapSite Abstract Base Class
class MapSite(ABC):
    @abstractmethod
    def Enter(self):
        pass

# Wall Class
class Wall(MapSite):
    def Enter(self):
        print("You just ran into a wall.")

# Room Class
class Room(MapSite):
    def __init__(self, roomNo: int):
        self._roomNumber = roomNo
        self._sides = [None] * 4

    def GetSide(self, direction: Direction) -> MapSite:
        return self._sides[direction.value]

    def SetSide(self, direction: Direction, site: MapSite):
        self._sides[direction.value] = site

    def Enter(self):
        print(f"Entering room {self._roomNumber}")

# Door Class
class Door(MapSite):
    def __init__(self, room1: Room = None, room2: Room = None):
        self._room1 = room1
        self._room2 = room2
        self._isOpen = False

    def Enter(self):
        if self._isOpen:
            print("You pass through the door.")
        else:
            print("The door is closed.")

    def OtherSideFrom(self, room: Room) -> Room:
        if room == self._room1:
            return self._room2
        elif room == self._room2:
            return self._room1
        else:
            return None
        
# Maze Class
class Maze:
    def __init__(self):
        self._rooms = {}

    def AddRoom(self, room: Room):
        self._rooms[room._roomNumber] = room

    def RoomNo(self, room_number: int) -> Room:
        return self._rooms.get(room_number)

## Hard-Coded Implementation

In [ ]:
class MazeGame:
    @staticmethod
    def CreateMaze() -> Maze:
        aMaze = Maze()
        r1 = Room(1)
        r2 = Room(2)
        theDoor = Door(r1, r2)
        
        aMaze.AddRoom(r1)
        aMaze.AddRoom(r2)
        
        r1.SetSide(Direction.NORTH, Wall())
        r1.SetSide(Direction.EAST, theDoor)
        r1.SetSide(Direction.SOUTH, Wall())
        r1.SetSide(Direction.WEST, Wall())
        
        r2.SetSide(Direction.NORTH, Wall())
        r2.SetSide(Direction.EAST, Wall())
        r2.SetSide(Direction.SOUTH, Wall())
        r2.SetSide(Direction.WEST, theDoor)
        
        return aMaze
    
maze = MazeGame.CreateMaze()
room1 = maze.RoomNo(1)
room2 = maze.RoomNo(2)

print(f"Room 1 sides: {[type(room1.GetSide(direction)).__name__ for direction in Direction]}")
print(f"Room 2 sides: {[type(room2.GetSide(direction)).__name__ for direction in Direction]}")

## Abstract Factory

Implementation of the class `MazeFactory` that can create the components of a maze.

The implmentation of `CreateMaze` allows creating mazes with different components by taking `MazeFactory` as parameter.

The `MazeFactory` is just a collection of factory methods. This is the most common way to implement the `Abstract Factory` pattern. 

In [ ]:
class MazeFactory:
    def MakeMaze():
        return Maze()
    
    def MakeWall():
        return Wall()
    
    def MakeRoom(n: int):
        return Room(n)
    
    def MakeDoor(r1: Room, r2: Room):
        return Door(r1, r2)

Note that `MazeFactory` is not an abstract class; thus it acts as both the `AbstractFactory` and the `ConcreteFactory`. This is another common implementation for simple applications of the `Abstract Factory` pattern. Because the `MazeFactory` is a concrete class consisting entirely of factory methods, it's easy to make a new `MazeFactory` by making a subclass and overriding the operations that need to change.

In [ ]:
class MazeGame:
    @staticmethod
    def CreateMaze(factory: MazeFactory) -> Maze:
        aMaze = factory.MakeMaze()
        r1 = factory.MakeRoom(1)
        r2 = factory.MakeRoom(2)
        aDoor = factory.MakeDoor(r1, r2)

        aMaze.AddRoom(r1)
        aMaze.AddRoom(r2)

        r1.SetSide(Direction.NORTH, factory.MakeWall())
        r1.SetSide(Direction.EAST, aDoor)
        r1.SetSide(Direction.SOUTH, factory.MakeWall())
        r1.SetSide(Direction.WEST, factory.MakeWall())
        
        r2.SetSide(Direction.NORTH, factory.MakeWall())
        r2.SetSide(Direction.EAST, factory.MakeWall())
        r2.SetSide(Direction.SOUTH, factory.MakeWall())
        r2.SetSide(Direction.WEST, aDoor)
        
        return aMaze

For example, it is possible to create `EnchantedMazeFactory`, that is a factory for enchanted mazes, by subclassing MazeFactory.

In [ ]:
class EnchantedRoom(Room):
    def __init__(self, roomNo, CastSpell):
        super().__init__(roomNo)

class DoorNeedingSpell(Door):
    def __init__(self, r1, r2):
        super().__init__(r1, r2)

class EnchantedMazeFactory(MazeFactory):
    def MakeRoom(self, n):
        return EnchantedRoom(n, self.CastSpell())
    
    def MakeDoor(self, r1:Room, r2:Room):
        return DoorNeedingSpell(r1, r2)
    
    def CastSpell(self):
        print("Entered Enchanted Room")

Now suppose we want to make a maze game in which a room can have a bomb set in it. If the bomb goes off, it will damage the walls (at least). We can make a subclass of Room keep track of whether the room has a bomb in it and whether the bomb has gone off. We'll also need a subclass of Wall to keep track of the damage done to the wall. We'll call these classes RoomWithABomb and BombedWall.

The last class we'll define is BombedMazeFactory, a subclass of MazeFactory that ensures walls are of class BombedWall and rooms are of class RoomWithABomb. BombedMazeFactory only needs to override two functions:

```cpp
Wall* BombedMazeFactory::MakeWall () const {
return new BombedWall;
}
Room* BombedMazeFactory::MakeRoom(int n) const {
return new RoomWithABomb(n);
}
```

To build a simple maze that can contain bombs, we simply call CreateMaze with a BombedMazeFactory.

```cpp
MazeGame game;
BombedMazeFactory factory;
game.CreateMaze(factory);
```

## Builder

In [ ]:
class MazeBuilder(ABC):
    def BuildMaze():
        pass
    def BuildRoom(room:int):
        pass
    def BuildDoor(roomFrom:int, roomTo: int):
        pass
    @abstractmethod
    def GetMaze():
        pass

class MazeGame:
    @staticmethod
    def CreateMaze(builder: MazeBuilder):
        builder.BuildMaze()
        builder.BuildRoom(1)
        builder.BuildRoom(2)
        builder.BuildDoor(1, 2)

        return builder.GetMaze()
